# Ricci Flow Clustering algorithm for high dimensional data

Based on: 
* https://github.com/saibalmars/GraphRicciCurvature/blob/master/notebooks/tutorial.ipynb
* https://www.nature.com/articles/s41598-019-46380-9 (graph clustering)
* https://arxiv.org/pdf/2303.02561.pdf (dim reduction)

There is a recent paper that does dimension reduction using Ricci flows, and it seems very promision. They start by constructing the weighted UMAP graph, and then they modify the weights using a notion of curvature. They compare their results against UMAP and what they get is almost too good to be true. Leland suggested to look into it in order to see if any ideas there could be apply to high dimensional clustering. It turns out that the paper they based their work off is a graph clustering paper that uses Ricci flows on graphs. Those results were published in Nature and their code is available.

Here, I try to run their graph clustering code on the UMAP graph we get and the results I get are juste terrible. François did the same on graphs very easy to cluster, and same results for him: nothing works.

*Their summary*:
By considering networks as geometric objects and communities in a network as a geometric
decomposition, we apply curvature and discrete Ricci flow, which have been used to decompose smooth manifolds with
astonishing successes in mathematics, to break down communities in networks. We apply this method to k-NN graphs obtained from high dimensional data.

In [1]:
!git branch

* master


In [2]:
execfile('functions/data_specifics.py')
execfile('functions/graph_functions.py')
print(data_set_list)

['pendigits', 'coil', 'mnist', 'usps', 'buildings', 'clusterable']


In [3]:
from IPython.display import display, Markdown, Latex
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score, silhouette_score
from sklearn import cluster

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import umap
from collections import Counter
from scipy.stats import mode

from scipy.spatial.distance import euclidean

sns.set()

In [4]:
import GraphRicciCurvature
print(GraphRicciCurvature.__version__)

0.5.3.1


In [5]:
import networkx as nx
import numpy as np
import math

# matplotlib setting
%matplotlib inline
import matplotlib.pyplot as plt

# to print logs in jupyter notebook
import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

# load GraphRicciCuravture package
from GraphRicciCurvature.OllivierRicci import OllivierRicci
from GraphRicciCurvature.FormanRicci import FormanRicci

# load python-louvain for modularity computation
import community as community_louvain

# for ARI computation
from sklearn import preprocessing, metrics

## Get data, UMAP graph and UMAP low dimensional vectors

In [24]:
dataset_id=0
raw_data, targets, dataset_name = get_dataset(dataset_id=dataset_id)

k = get_dataset_params(dataset_id)['n_neighbors']

In [16]:
# print(f'We are building a {k}-NN graph for dataset {dataset_name}')
# A_umap, sigmas, rhos, dists = umap.umap_.fuzzy_simplicial_set(X=raw_data, 
#                                              n_neighbors=k, 
#                                              random_state=0, 
#                                              metric='euclidean', 
#                                              return_dists=True,
#                                              set_op_mix_ratio=1)
# umap_rep = get_umap_vectors(dataset_id=dataset_id, raw_data=raw_data)

# A = (A_umap > 0)
# G = nx.from_scipy_sparse_matrix(A)

In [25]:
print(f'We are building a directed {k}-NN graph for dataset {dataset_name}')
G = knn_digraph(raw_data, k=k, graph_type='nx')

We are building a directed 15-NN graph for dataset pendigits


In [30]:
print("\n=====  Compute Ricci flow metric - Optimal Transportation Distance =====")

orc_OTD = OllivierRicci(G, alpha=0.5, method="OTD", verbose="INFO")
orc_OTD.compute_ricci_flow(iterations=10)


=====  Compute Ricci flow metric - Optimal Transportation Distance =====


2023-03-21 16:36:45,650 - OllivierRicci - INFO - No ricciCurvature detected, compute original_RC...
2023-03-21 16:36:46,666 - OllivierRicci - INFO - 0.583161 secs for Ricci curvature computation.
2023-03-21 16:36:46,927 - OllivierRicci - INFO -  === Ricci flow iteration 0 === 
2023-03-21 16:36:47,933 - OllivierRicci - INFO - 0.561057 secs for Ricci curvature computation.
2023-03-21 16:36:48,183 - OllivierRicci - INFO -  === Ricci flow iteration 1 === 
2023-03-21 16:36:49,180 - OllivierRicci - INFO - 0.564841 secs for Ricci curvature computation.
2023-03-21 16:36:49,432 - OllivierRicci - INFO -  === Ricci flow iteration 2 === 
2023-03-21 16:36:50,432 - OllivierRicci - INFO - 0.570502 secs for Ricci curvature computation.
2023-03-21 16:36:50,683 - OllivierRicci - INFO -  === Ricci flow iteration 3 === 
2023-03-21 16:36:51,677 - OllivierRicci - INFO - 0.564937 secs for Ricci curvature computation.
2023-03-21 16:36:51,928 - OllivierRicci - INFO -  === Ricci flow iteration 4 === 
2023-03-21

In [27]:
%%time
print("\n=====  Compute Ricci community - by Ricci flow =====")
clustering = orc_OTD.ricci_community()

2023-03-21 16:17:57,616 - OllivierRicci - INFO - Ricci flow not detected yet, run Ricci flow with default setting first...
2023-03-21 16:17:57,642 - OllivierRicci - INFO - No ricciCurvature detected, compute original_RC...



=====  Compute Ricci community - by Ricci flow =====


2023-03-21 16:17:58,662 - OllivierRicci - INFO - 0.586883 secs for Ricci curvature computation.
2023-03-21 16:17:58,918 - OllivierRicci - INFO -  === Ricci flow iteration 0 === 
2023-03-21 16:18:01,286 - OllivierRicci - INFO - 1.216654 secs for Ricci curvature computation.
2023-03-21 16:18:02,054 - OllivierRicci - INFO -  === Ricci flow iteration 1 === 
2023-03-21 16:18:04,257 - OllivierRicci - INFO - 1.366657 secs for Ricci curvature computation.
2023-03-21 16:18:04,908 - OllivierRicci - INFO -  === Ricci flow iteration 2 === 
2023-03-21 16:18:07,084 - OllivierRicci - INFO - 1.266373 secs for Ricci curvature computation.
2023-03-21 16:18:07,573 - OllivierRicci - INFO -  === Ricci flow iteration 3 === 
2023-03-21 16:18:09,470 - OllivierRicci - INFO - 1.065014 secs for Ricci curvature computation.
2023-03-21 16:18:10,040 - OllivierRicci - INFO -  === Ricci flow iteration 4 === 
2023-03-21 16:18:12,504 - OllivierRicci - INFO - 1.630779 secs for Ricci curvature computation.
2023-03-21 16:

CPU times: user 2min 36s, sys: 7.42 s, total: 2min 44s
Wall time: 2min 8s


In [28]:
labels = list(clustering[1].values())
ari = adjusted_rand_score(targets, labels)
ami = adjusted_mutual_info_score(targets, labels)
print(f'ARI = {ari} and AMI = {ami}') 

ARI = -0.003938781961610883 and AMI = -0.008027498514716966
